### <span style="color:red"><b>Spotify Music Recommendation System---Using Spotify Api</b></span>

In [1]:
import requests
import base64
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

#### <span style="color:green"><i>Users Credentials Generated using Spotify Developer</i></span>

In [2]:
CLIENT_ID ='972aa9d5a7ac4100994898a47b683ee3'
CLIENT_SECRET = '7152374d0eae4fbfb762e792781f72ea'

In [3]:
CLIENT_CREDENTIALS =f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(CLIENT_CREDENTIALS.encode())

####  <span style="color:green"><i>Accessing Token From Spotify API</i></span>

In [4]:
token_url = 'https://accounts.spotify.com/api/token'
headers={
    'Authorization':f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type':'client_credentials'

}
response = requests.post(token_url,headers=headers,data=data)
if response.status_code == 200:
    access_token = response.json()['access_token']
    m=access_token
else:
    print('Error:', response.status_code, response.text)

####  <span style="color:green"><i>Retrieving Playlist Information</i></span>

In [5]:
def get_trending_playlist_data(playlist_id, access_token):
    sp = spotipy.Spotify(auth=access_token)
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    music_data = []

    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artist_names = [artist['name'] for artist in track['artists']]
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        track_data = {
            'Track Name': track_name,
            'Artist': ', '.join(artist_names),
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Release Date': release_date,
            'Popularity': popularity,
            'Duration(ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None) if track_info else None,
            'External URLs': track_info.get('external_urls', {}).get('spotify', None) if track_info else None,
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)

    df = pd.DataFrame(music_data)
    return df


####  <span style="color:green"><i>Intialising Playlist_id and Generating Tracks Info</i></span>

In [7]:
playlist_id = '44j7hYaG0v6DEh9egqVWzw'
access_token=m


# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df.head(5)

,Track Name,Artist,Album Name,Album ID,Track ID,Release Date,Popularity,Duration(ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Summer of Love (Shawn Mendes & Tainy),"Shawn Mendes, Tainy",Summer Of Love,4nProAxFavzKkbaRQ2Otyk,0z8hI3OPS8ADPWtoCjjLl6,2021-08-20,65,184104,False,https://open.spotify.com/track/0z8hI3OPS8ADPWt...,...,0.808,11,-4.501,1,0.1170,0.0297,0.000127,0.1030,0.494,123.988
1,STAY (with Justin Bieber),"The Kid LAROI, Justin Bieber",STAY (with Justin Bieber),4QLAtpLNUsHEYrcHXmMIZZ,5HCyWlXZPP0y6Gqq8TgA20,2021-07-09,76,141806,True,https://open.spotify.com/track/5HCyWlXZPP0y6Gq...,...,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928
2,Shape of You,Ed Sheeran,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,7qiZfU4dY1lWllzX7mPBI3,2017-03-03,81,233713,False,https://open.spotify.com/track/7qiZfU4dY1lWllz...,...,0.652,1,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977
3,Peaches (feat. Daniel Caesar & Giveon),"Justin Bieber, Daniel Caesar, Giveon",Justice,5dGWwsZ9iB2Xc3UKR0gif2,4iJyoBOLtHqaGxP12qzhQI,2021-03-19,75,198082,True,https://open.spotify.com/track/4iJyoBOLtHqaGxP...,...,0.696,0,-6.181,1,0.1190,0.3210,0.000000,0.4200,0.464,90.030
4,Bad Habits,Ed Sheeran,=,32iAEBstCjauDhyKpGjTuq,3rmo8F54jFF8OgYsqTxm5d,2021-10-25,76,230747,False,https://open.spotify.com/track/3rmo8F54jFF8OgY...,...,0.893,11,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011


#### <span style="color:green"><i>Checking for Nulll Values</i></span>

In [8]:
music_df.isnull().sum()

Track Name          0
Artist              0
Album Name          0
Album ID            0
Track ID            0
Release Date        0
Popularity          0
Duration(ms)        0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [9]:
data=music_df

### <span><b> Building Hybrid Recommendation System</b></span>

#### <span style="color:green"><i>Giving Weights To The Latest Releases</i></span>

In [10]:
def calculate_weighted_popularity(release_date):
  release_date=datetime.strptime(release_date,'%Y-%m-%d')
  time_span=datetime.now()-release_date
  days=time_span.days
  weight=1/(time_span.days+1)
  return weight

 #### <span style="color:green"><i>Normalizing Music Features</i></span>

In [11]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

#### <span style="color:Green"><i>Recommending Music -- Based On Music Audio Feature</i></span> 

In [12]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return None

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations+1]

    content_based_recommendation = music_df.iloc[similar_song_indices][['Track Name', 'Artist', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendation

#### <span style="color:green"><i>Recommending Music -- Based On Weighted Popularity</i></span>

In [13]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return None

    
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)
    if content_based_rec is None:
        return None

    
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]


    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

 
    input_song_data = pd.DataFrame([{
        'Track Name': input_song_name,
        'Artist': music_df.loc[music_df['Track Name'] == input_song_name, 'Artist'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }])

    hybrid_recommendations = pd.concat([content_based_rec, input_song_data], ignore_index=True)


    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)


    hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
    
    return hybrid_recommendations

#### <span style="color:green"><b><i>Generating Music recommendations</i></b></span>

In [14]:
print("Search Your Tunes")
input_song_name = input()
recommendations =hybrid_recommendations(input_song_name, num_recommendations=5)
recommendations.head()

Search Your Tunes


 Peaches (feat. Daniel Caesar & Giveon)


,Track Name,Artist,Album Name,Release Date,Popularity
2,Save Your Tears,The Weeknd,After Hours,2020-03-20,80.0
0,Dura,Daddy Yankee,Dura,2018-01-18,65.0
4,SugarCrash!,ElyOtto,SugarCrash!,2020-08-25,64.0
1,What If (I Told You I Like You),"Johnny Orlando, kenzie",What If (I Told You I Like You),2018-05-18,58.0
3,Jesus in LA,Alec Benjamin,These Two Windows,2020-05-29,56.0
